# Test data

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
sheets = pd.ExcelFile('Test.xlsx')

In [3]:
sheet_dfs = {}
for sheet in sheets.sheet_names:
    df = sheets.parse(sheet)
    df.set_index(df.columns[0], inplace=True)
    sheet_dfs[sheet] = df

In [4]:
attributes = sheet_dfs['attributes']

In [5]:
importance_sum = attributes['Importance'].sum(axis=0)

In [6]:
importance = attributes['Importance']/importance_sum

In [7]:
attributes_norm = attributes.drop('Importance',axis=1).T

In [8]:
scaler = MinMaxScaler()
X = scaler.fit_transform(attributes_norm)
X = pd.DataFrame(X, index=attributes_norm.index, columns=attributes_norm.columns)
X = X.T
X.columns = [col + '_norm' for col in X.columns]

In [9]:
coefficient = {}
for col in X.columns:
    product = X[col]*importance
    coefficient[col] = product.sum(axis=0)

In [10]:
distance = sheet_dfs['distance']

In [11]:
preference_df = pd.DataFrame([], index=distance.index)
for col in distance.columns:
    preference_df[col+'_pref'] = coefficient[col+'_norm']/(distance[col])**2

In [12]:
pref_sum = preference_df.sum(axis=1)

In [13]:
prob_df = pd.DataFrame([],index=preference_df.index)

In [14]:
for col in preference_df.columns:
    prob_df[col+'_pr'] = preference_df[col]/pref_sum

In [15]:
communities = sheet_dfs['communities']

In [16]:
comm_expenditure = communities['Number of households'] * communities['Average grocery exp']

In [17]:
expenditure_df = pd.DataFrame([],index=prob_df.index)
for col in prob_df.columns:
    expenditure_df[col.split('_')[0]] = prob_df[col]*comm_expenditure

In [18]:
expense = expenditure_df.sum(axis=0).to_dict()

In [19]:
preferred_store = [key for key in expense.keys() if expense[key] == max(expense.values())]

In [20]:
f'The preferred store location is {preferred_store[0]}'

'The preferred store location is Y'

# Trade Area Modelling Assignment

In [2]:
sheets = pd.ExcelFile('Trade_Area_assignment.xlsx')

In [3]:
sheet_dfs = {}

In [4]:
for sheet in sheets.sheet_names:
    df = sheets.parse(sheet)
    df.columns = df.columns.str.strip()
    for col in df.columns:
        try: df[col] = df[col].str.strip()
        except: pass
    df.set_index(df.columns[0], inplace=True)
    sheet_dfs[sheet] = df

In [5]:
sheet_dfs.keys()

dict_keys(['Distance', 'Census', 'stores'])

In [6]:
attributes = sheet_dfs['stores']

In [7]:
scaler = MinMaxScaler()
X = scaler.fit_transform(attributes)
X = pd.DataFrame(X, index=attributes.index, columns=attributes.columns)

In [8]:
X_sum = X.sum(axis=1)

In [12]:
distance = sheet_dfs['Distance']

In [13]:
store_pref, prob_df, expense_df = (pd.DataFrame([],index=distance.index) for _ in range(3))

In [15]:
for col in distance.columns:
    store_pref[col] = X_sum[col]/(distance[col])**2

In [19]:
pref_sum = store_pref.sum(axis=1)

In [20]:
for col in store_pref.columns:
    prob_df[col] = store_pref[col]/pref_sum

In [21]:
communities = sheet_dfs['Census']

In [28]:
expense = communities['Households'] * communities['Expenditure grocery']

In [29]:
expense

customer community
Adams                          3200000
Arleta                         8000000
Atwater Village               13200000
Barnes City                    4200000
Bel Air                        1200000
Benedict-Coldwater Canyons     3500000
Echo Park                      6300000
San Pedro                      4500000
Sherman Oaks                   5600000
Granada Hills                  6183574
Lakeview Terrace               6873750
Panorama City                  6388756
Toluca Lake                   11364948
Warner Center                 15721350
Wholesale District            15566029
dtype: int64

In [23]:
for col in prob_df.columns:
    expense_df[col] = prob_df[col]*expense

In [24]:
expense = expense_df.sum(axis=0).sort_values(ascending=False).to_dict()

In [25]:
preferred_store = [key for key in expense.keys() if expense[key] == max(expense.values())]

In [26]:
f'The preferred store location is {preferred_store[0]}'

'The preferred store location is Lakeview Terrace'